# 0. Import Packages

In [17]:
import pandas as pd
import os
import numpy as np

# 1. Process Data
(skip this step if data is already saved)

In [18]:
def build_cpc_field_slice_dicts():
    g = [
        ('kind',                (0 ,2 )),   
        ('application_number',  (2 ,10)),  
        ('document_number',     (10,18)),
        ('cpc_section',         (18,19)), 
        ('cpc_class',           (18,21)), 
        ('cpc_subclass',        (18,22)), 
        ('cpc_main_group',      (18,26)), 
        ('cpc_subgroup',        (18,33)), 
        ('cpc_version_date',    (33,41)), 
        ('cpc_symbol_position', (41,42)), 
        ('cpc_value_code',      (42,43)), 
        ('cpc_set_group',       (43,46)), 
        ('cpc_set_rank',        (46,48))]
    a = g[:2] + [(k,(v[0],v[1]+3)) for k,v in g[2:3]] + \
            [(k,(v[0]+3,v[1]+3)) for k,v in g[3:]]
    return {'grant': g, 'application': a}

In [19]:
cpc_field_slice_dict = build_cpc_field_slice_dicts()

In [20]:
def process_cpc_classification_file(textfile, document_type):

    d = pd.read_fwf(
            textfile,
            colspecs=[i[1] for i in cpc_field_slice_dict[document_type]],
            names=[i[0] for i in cpc_field_slice_dict[document_type]],
            dtype=str)
    d['cpc_version_date'] = pd.to_datetime(
            d['cpc_version_date'], yearfirst=True)
    return d

In [21]:
def process_cpc_classification_folder(directory, document_type):
    
    master_df = []
    counter = 0
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        d = process_cpc_classification_file(file_path, document_type)
        master_df.append(d)
        counter += 1
        print(counter)
    return pd.concat(master_df, ignore_index=True)

In [22]:
df_grant = process_cpc_classification_folder("US_Grant_CPC_MCF_Text_2020-01-01", "grant")

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211


In [24]:
# idea: train on grant and test on applications?
df_grant.to_feather('df_grant.ft')

# 2. Merge with API Data

In [1]:
import feather
df_grant = feather.read_dataframe('df_grant.ft')

In [3]:
df_grant.tail()

,kind,application_number,document_number,cpc_section,cpc_class,cpc_subclass,cpc_main_group,cpc_subgroup,cpc_version_date,cpc_symbol_position,cpc_value_code,cpc_set_group,cpc_set_rank
49298109,B2,10429179,6799998,Y,10,T,29,49222,2015-01-15,L,A,0,0
49298110,B2,10359843,6799999,H,01,R,13,7195,2013-01-01,F,I,0,0
49298111,B2,10359843,6799999,H,01,R,4,185,2013-01-01,L,A,0,0
49298112,B2,10359843,6799999,H,01,R,13,112,2013-01-01,L,A,0,0
49298113,B2,10359843,6799999,H,01,R,13,506,2013-01-01,L,A,0,0


In [5]:
df_grant['document_number_long'] = 'US' + df_grant['document_number'] +df_grant['kind']

In [6]:
df_grant['document_number_long']

0           US9050000B2
1           US9050000B2
2           US9050000B2
3           US9050000B2
4           US9050000B2
               ...     
49298109    US6799998B2
49298110    US6799999B2
49298111    US6799999B2
49298112    US6799999B2
49298113    US6799999B2
Name: document_number_long, Length: 49298114, dtype: object

In [7]:
df_valid = pd.read_table("3M.doc.ids.txt", header = None)
df_valid.columns = ["raw_id"]

In [8]:
len(df_valid)

3819401

In [9]:
df_valid.head()

,raw_id
0,US10406442B2
1,US10406441B2
2,US10406440B2
3,US10406439B1
4,US10406438B2


In [10]:
# how long are the raw document IDs from API?
df_valid['length'] = df_valid["raw_id"].str.len()
df_valid['length'].value_counts()

11    1911501
10    1470158
12     437741
6           1
Name: length, dtype: int64

In [11]:
# length = 10 (usually ends with a single letter A)
df_valid[df_valid.length == 10].head()

,raw_id,length
90100,US6018475A,10
90102,US6018474A,10
90104,US6018473A,10
90106,US6018472A,10
90108,US6018471A,10


In [12]:
# length = 11 (usually ends with a single letter and a digit)
df_valid[df_valid.length == 11].head()

,raw_id,length
40000,US6929031B2,11
40001,US6929030B2,11
40002,US6929029B1,11
40003,US6929028B2,11
40004,US6929027B2,11


In [13]:
# length = 12 (usually ends with a single letter and a digit)
df_valid[df_valid.length == 12].head()

,raw_id,length
0,US10406442B2,12
1,US10406441B2,12
2,US10406440B2,12
3,US10406439B1,12
4,US10406438B2,12


In [15]:
df_grant["text_available"] = np.where(df_grant["document_number_long"].isin(df_valid["raw_id"]), 1, 0)

In [16]:
df_grant_clean = df_grant[df_grant["text_available"] == 1]

In [17]:
df_grant["document_number_len"] = df_grant["document_number"].str.len()

In [20]:
df_grant["document_number_len"].value_counts()

7    43093337
8     4840310
6     1229210
5      121592
4       12234
3        1277
2         134
1          20
Name: document_number_len, dtype: int64

# 3. Data Analysis

## 2.1 Summay statistics

In [21]:
# total number of rows
print(len(df_grant))
print(len(df_grant_clean))

49298114
13745660


In [22]:
# number of unique grants
print(len(df_grant.document_number.drop_duplicates()))
print(len(df_grant_clean.document_number.drop_duplicates()))

10465351
2359350


## 2.2 Investigate CPC version date

In [23]:
# investigate issues with multiple cpc_version_date
doc_date_dedup = df_grant[["document_number", "cpc_version_date"]].drop_duplicates()
doc_date_dedup_value_counts = doc_date_dedup["document_number"].value_counts()

In [24]:
doc_date_dedup_value_counts.value_counts(normalize=True).mul(100).round(1).astype(str)+"%"

1    80.3%
2    17.3%
3     2.1%
4     0.2%
5     0.0%
6     0.0%
7     0.0%
8     0.0%
9     0.0%
Name: document_number, dtype: object

In [25]:
doc_date_dedup[doc_date_dedup.document_number == "6030673"]

,document_number,cpc_version_date
19772069,6030673,2013-01-01
19772103,6030673,2016-11-01
19772169,6030673,2018-08-01
19772178,6030673,2019-02-01
19772201,6030673,2017-08-01
19772215,6030673,2018-01-01
19772216,6030673,2015-05-01
19772219,6030673,2015-01-15
19772223,6030673,2015-04-01


In [26]:
df_grant[df_grant.document_number == "6030673"].sort_values(by=['cpc_version_date'], ascending = False)

,kind,application_number,document_number,cpc_section,cpc_class,cpc_subclass,cpc_main_group,cpc_subgroup,cpc_version_date,cpc_symbol_position,cpc_value_code,cpc_set_group,cpc_set_rank,document_number_long,text_available,document_number_len
19772179,A,09246163,6030673,B,29,C,48,0017,2019-02-01,L,A,0,0,US6030673A,1,7
19772178,A,09246163,6030673,B,29,C,48,0011,2019-02-01,L,A,0,0,US6030673A,1,7
19772180,A,09246163,6030673,B,29,C,48,03,2019-02-01,L,A,0,0,US6030673A,1,7
19772181,A,09246163,6030673,B,29,C,48,15,2019-02-01,L,A,0,0,US6030673A,1,7
19772169,A,09246163,6030673,B,05,B,11,0038,2018-08-01,L,A,0,0,US6030673A,1,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19772145,A,09246163,6030673,C,04,B,26,285,2013-01-01,L,I,0,0,US6030673A,1,7
19772146,A,09246163,6030673,C,04,B,28,00,2013-01-01,L,I,0,0,US6030673A,1,7
19772147,A,09246163,6030673,C,04,B,28,02,2013-01-01,L,I,0,0,US6030673A,1,7
19772148,A,09246163,6030673,C,04,B,38,0006,2013-01-01,L,I,0,0,US6030673A,1,7


In [27]:
# are there duplicates in terms of version date? keep the latest date? Probably no

## 2.3 Investigate multi-label problem

In [38]:
def report_freq(series):
    return pd.concat([series.value_counts(),
                      series.value_counts(normalize=True).mul(100)
                      .round(1).astype(str)+"%"], axis=1, keys=['count', '%'])

In [57]:
def report_freq_compare(left_series, right_series):
    left_freq = report_freq(left_series)
    right_freq = report_freq(right_series)
    return left_freq.merge(right_freq, how='outer', left_index=True, right_index=True, suffixes=('_all', '_clean'))

In [59]:
# section level
multilabel_section = df_grant[["document_number", 
                               "cpc_section"]].drop_duplicates()["document_number"].value_counts()
multilabel_section_clean = df_grant_clean[["document_number", 
                               "cpc_section"]].drop_duplicates()["document_number"].value_counts()
report_freq_compare(multilabel_section, multilabel_section_clean)

,count_all,%_all,count_clean,%_clean
1,6937527,66.3%,1445175.0,61.3%
2,2856550,27.3%,708927.0,30.0%
3,579848,5.5%,176657.0,7.5%
4,82090,0.8%,25746.0,1.1%
5,8450,0.1%,2574.0,0.1%
6,805,0.0%,250.0,0.0%
7,75,0.0%,21.0,0.0%
8,1,0.0%,NaN,NaN
9,5,0.0%,NaN,NaN


In [60]:
# subclass level
report_freq_compare(df_grant[["document_number", "cpc_section", "cpc_class", "cpc_subclass"]]
                    .drop_duplicates()["document_number"].value_counts(),
                    df_grant_clean[["document_number", "cpc_section", "cpc_class", "cpc_subclass"]]
                    .drop_duplicates()["document_number"].value_counts())

,count_all,%_all,count_clean,%_clean
1,5774897,55.2%,1123232.0,47.6%
2,2868828,27.4%,703955.0,29.8%
3,1113743,10.6%,317616.0,13.5%
4,420439,4.0%,127785.0,5.4%
5,166647,1.6%,50699.0,2.1%
6,68274,0.7%,20354.0,0.9%
7,27892,0.3%,8374.0,0.4%
8,12043,0.1%,3601.0,0.2%
9,5571,0.1%,1660.0,0.1%
10,2782,0.0%,774.0,0.0%


In [61]:
# subgroup level
report_freq_compare(df_grant["document_number"].value_counts(),
                    df_grant_clean["document_number"].value_counts())

,count_all,%_all,count_clean,%_clean
1,3227654,30.8%,386468.0,16.4%
2,1811974,17.3%,407162.0,17.3%
3,1238930,11.8%,329467.0,14.0%
4,893880,8.5%,255137.0,10.8%
5,663202,6.3%,194523.0,8.2%
...,...,...,...,...
627,1,0.0%,NaN,NaN
735,1,0.0%,NaN,NaN
853,1,0.0%,1.0,0.0%
1107,2,0.0%,1.0,0.0%


## 2.4 Investigate unique values at different levels

In [46]:
# unique values at different levels
levels = ["cpc_section", "cpc_class", "cpc_subclass", "cpc_main_group", "cpc_subgroup"]

In [47]:
subgroups = df_grant[levels].drop_duplicates()
counter = 1
for level in levels:
    print(level + ": " + str(len(subgroups[levels[:counter]].drop_duplicates())))
    counter += 1

cpc_section: 9
cpc_class: 129
cpc_subclass: 666
cpc_main_group: 10661
cpc_subgroup: 246394


In [48]:
subgroups = df_grant_clean[levels].drop_duplicates()
counter = 1
for level in levels:
    print(level + ": " + str(len(subgroups[levels[:counter]].drop_duplicates())))
    counter += 1

cpc_section: 9
cpc_class: 129
cpc_subclass: 664
cpc_main_group: 10359
cpc_subgroup: 228082


## 2.5 Investigate data imbalance problem

In [62]:
# distribution at the section level
report_freq_compare(df_grant["cpc_section"], df_grant_clean["cpc_section"])

,count_all,%_all,count_clean,%_clean
H,12151952,24.6%,3419275,24.9%
G,7831644,15.9%,2131574,15.5%
B,7287726,14.8%,2035631,14.8%
C,6605919,13.4%,1941242,14.1%
A,6190708,12.6%,1794726,13.1%
Y,4185576,8.5%,1118927,8.1%
F,3529646,7.2%,955299,6.9%
E,1014152,2.1%,244926,1.8%
D,500791,1.0%,104060,0.8%


In [67]:
def report_level_count(level_index, dataset):
    level_counts = dataset.groupby(levels[:level_index]).size().reset_index().rename(columns={0: "count"})
    level_counts = level_counts.sort_values(by=["count"], ascending = False)
    level_counts["percent"] = (level_counts["count"] / level_counts["count"].sum()).mul(100).round(1).astype(str)+"%"
    return level_counts

In [69]:
# distribution at the subclass level
report_level_count(3, df_grant)

,cpc_section,cpc_class,cpc_subclass,count,percent
611,H,01,L,4725004,9.6%
665,Y,10,T,2522801,5.1%
559,G,06,F,1819113,3.7%
641,H,04,L,1422524,2.9%
643,H,04,N,1246901,2.5%
...,...,...,...,...,...
492,F,24,J,38,0.0%
557,G,06,D,30,0.0%
603,G,21,Y,9,0.0%
647,H,04,T,4,0.0%


In [70]:
report_level_count(3, df_grant_clean)

,cpc_section,cpc_class,cpc_subclass,count,percent
610,H,01,L,1438444,10.5%
663,Y,10,T,643611,4.7%
559,G,06,F,421707,3.1%
640,H,04,L,355216,2.6%
68,A,61,K,346994,2.5%
...,...,...,...,...,...
601,G,21,J,10,0.0%
318,C,12,L,10,0.0%
251,B,68,F,5,0.0%
557,G,06,D,3,0.0%


In [72]:
# distribution at the subgroup level
subgroup_counts = report_level_count(5, df_grant)
subgroup_counts

,cpc_section,cpc_class,cpc_subclass,cpc_main_group,cpc_subgroup,count,percent
205653,H,01,L,2924,00,213944,0.4%
205658,H,01,L,2924,00014,194991,0.4%
205660,H,01,L,2924,0002,116103,0.2%
20240,A,61,K,2300,00,99818,0.2%
202800,H,01,L,2224,48091,69536,0.1%
...,...,...,...,...,...,...,...
128129,E,05,Y,2201,61,1,0.0%
87266,C,03,C,2201,3452,1,0.0%
52765,B,29,C,66,81885,1,0.0%
180556,G,05,B,2219,50105,1,0.0%


In [73]:
subgroup_counts_clean = report_level_count(5, df_grant_clean)
subgroup_counts_clean

,cpc_section,cpc_class,cpc_subclass,cpc_main_group,cpc_subgroup,count,percent
189458,H,01,L,2924,00,69095,0.5%
189463,H,01,L,2924,00014,61283,0.4%
189465,H,01,L,2924,0002,37686,0.3%
19178,A,61,K,2300,00,30812,0.2%
186990,H,01,L,2224,48091,23321,0.2%
...,...,...,...,...,...,...,...
209805,H,05,B,41,23,1,0.0%
164999,G,05,B,2219,39084,1,0.0%
209803,H,05,B,41,20,1,0.0%
165003,G,05,B,2219,39094,1,0.0%


In [74]:
len(subgroup_counts)

246394

In [75]:
len(subgroup_counts[subgroup_counts["count"] >= 10])

209439

In [76]:
len(subgroup_counts[subgroup_counts["count"] >= 100])

100365

# 3. Generate Tree Structure

In [25]:
levels = ["cpc_section", "cpc_class", "cpc_subclass", "cpc_main_group", "cpc_subgroup"]
subgroups = df_grant[levels].drop_duplicates()

In [26]:
subgroups.head()

,cpc_section,cpc_class,cpc_subclass,cpc_main_group,cpc_subgroup
0,A,A61,A61B,A61B 5,A61B 5/02444
1,A,A61,A61B,A61B 5,A61B 5/6887
2,A,A61,A61B,A61B 5,A61B 5/6892
3,A,A61,A61B,A61B 5,A61B 5/0816
4,A,A61,A61B,A61B2562,A61B2562/0247


In [42]:
def create_tree(subgroups):
    tree = {'Root': dict()}
    for _, row in subgroups.iterrows():
        cpc_section, cpc_class, cpc_subclass, cpc_main_group, cpc_subgroup = row[levels]
        current_dict = tree['Root']
        
        # section
        if cpc_section not in current_dict:
            current_dict[cpc_section] = dict()
        current_dict = current_dict[cpc_section] 
        
        # class
        if cpc_class not in current_dict:
            current_dict[cpc_class] = dict()
        current_dict = current_dict[cpc_class] 
            
        # subclass
        if cpc_subclass not in current_dict:
            current_dict[cpc_subclass] = dict()
        current_dict = current_dict[cpc_subclass] 
        
        # main group and subgroup
        if cpc_main_group not in current_dict:
            current_dict[cpc_main_group] = set()
        current_dict[cpc_main_group].add(cpc_subgroup)
    return tree

In [43]:
tree = create_tree(subgroups)

In [47]:
import pickle
with open("cpc_label_tree.pkl","wb") as f:
    pickle.dump(tree, f)